# Multi-group Cross Section Generation

This notebook introduces the use of the `openmc.mgxs` module to calculate multi-group cross sections over a domain. In particular, we go over the following features:

* **General equations** for scalar-flux averaged multi-group cross sections
* Creation of multi-group cross sections for an **infinite homogeneous medium**
* Use of **tally arithmetic** to manipulate multi-group cross sections

Many Monte Carlo particle transport codes, including OpenMC, use continuous-energy nuclear cross section data. However, most deterministic neutron transport codes use *multi-group cross sections* defined over discretized energy bins or *energy groups*.

A variety of tools employing different methodologies have been developed over the years to compute multi-group cross sections for certain applications, including [NJOY](https://njoy.github.io/NJOY2016/), MC$^2$-3, and [Serpent](http://montecarlo.vtt.fi/). The `openmc.mgxs` Python module is designed to leverage OpenMC's tally system to calculate multi-group cross sections with arbitrary energy discretizations for fine-mesh heterogeneous deterministic neutron transport applications.

Before proceeding to illustrate how one may use the `openmc.mgxs` module, it is worthwhile to define the general equations used to calculate multi-group cross sections. This is only intended as a brief overview of the methodology used by `openmc.mgxs` - we refer the interested reader to the large body of literature on the subject for a more comprehensive understanding of this complex topic.

### Introductory Notation
The continuous real-valued microscopic cross section may be denoted $\sigma_{n,x}(\mathbf{r}, E)$ for position vector $\mathbf{r}$, energy $E$, nuclide $n$ and interaction type $x$. Similarly, the scalar neutron flux may be denoted by $\Phi(\mathbf{r},E)$ for position $\mathbf{r}$ and energy $E$. **Note**: Although nuclear cross sections are dependent on the temperature $T$ of the interacting medium, the temperature variable is neglected here for brevity.

### Spatial and Energy Discretization
The energy domain for critical systems such as thermal reactors spans more than 10 orders of magnitude of neutron energies from 10$^{-5}$ - 10$^7$ eV. The multi-group approximation discretization divides this energy range into one or more energy groups. In particular, for $G$ total groups, we denote an energy group index $g$ such that $g \in \{1, 2, ..., G\}$. The energy group indices are defined such that the smaller group the higher the energy, and vice versa. The integration over neutron energies across a discrete energy group is commonly referred to as **energy condensation**.

Multi-group cross sections are computed for discretized spatial zones in the geometry of interest. The spatial zones may be defined on a structured and regular fuel assembly or pin cell mesh, an arbitrary unstructured mesh or the constructive solid geometry used by OpenMC. For a geometry with $K$ distinct spatial zones, we designate each spatial zone an index $k$ such that $k \in \{1, 2, ..., K\}$. The volume of each spatial zone is denoted by $V_{k}$. The integration over discrete spatial zones is commonly referred to as **spatial homogenization**.

### General Scalar-Flux Weighted MGXS
The multi-group cross sections computed by `openmc.mgxs` are defined as a *scalar flux-weighted average* of the microscopic cross sections across each discrete energy group. This formulation is employed in order to preserve the reaction rates within each energy group and spatial zone. In particular, spatial homogenization and energy condensation are used to compute the general multi-group cross section $\sigma_{n,x,k,g}$ as follows:

$$\sigma_{n,x,k,g} = \frac{\int_{E_{g}}^{E_{g-1}}\mathrm{d}E'\int_{\mathbf{r} \in V_{k}}\mathrm{d}\mathbf{r}\sigma_{n,x}(\mathbf{r},E')\Phi(\mathbf{r},E')}{\int_{E_{g}}^{E_{g-1}}\mathrm{d}E'\int_{\mathbf{r} \in V_{k}}\mathrm{d}\mathbf{r}\Phi(\mathbf{r},E')}$$

This scalar flux-weighted average microscopic cross section is computed by `openmc.mgxs` for most multi-group cross sections, including total, absorption, and fission reaction types. These double integrals are stochastically computed with OpenMC's tally system - in particular, [filters](https://mit-crpg.github.io/openmc/pythonapi/filter.html) on the energy range and spatial zone (material, cell or universe) define the bounds of integration for both numerator and denominator.

### Multi-Group Scattering Matrices
The general multi-group cross section $\sigma_{n,x,k,g}$ is a vector of $G$ values for each energy group $g$. The equation presented above only discretizes the energy of the incoming neutron and neglects the outgoing energy of the neutron (if any). Hence, this formulation must be extended to account for the outgoing energy of neutrons in the discretized scattering matrix cross section used by deterministic neutron transport codes. 

We denote the incoming and outgoing neutron energy groups as $g$ and $g'$ for the microscopic scattering matrix cross section $\sigma_{n,s}(\mathbf{r},E)$. As before, spatial homogenization and energy condensation are used to find the multi-group scattering matrix cross section $\sigma_{n,s,k,g \to g'}$ as follows:

$$\sigma_{n,s,k,g\rightarrow g'} = \frac{\int_{E_{g'}}^{E_{g'-1}}\mathrm{d}E''\int_{E_{g}}^{E_{g-1}}\mathrm{d}E'\int_{\mathbf{r} \in V_{k}}\mathrm{d}\mathbf{r}\sigma_{n,s}(\mathbf{r},E'\rightarrow E'')\Phi(\mathbf{r},E')}{\int_{E_{g}}^{E_{g-1}}\mathrm{d}E'\int_{\mathbf{r} \in V_{k}}\mathrm{d}\mathbf{r}\Phi(\mathbf{r},E')}$$

This scalar flux-weighted multi-group microscopic scattering matrix is computed using OpenMC tallies with both energy in and energy out filters.

### Multi-Group Fission Spectrum
The energy spectrum of neutrons emitted from fission is denoted by $\chi_{n}(\mathbf{r},E' \rightarrow E'')$ for incoming and outgoing energies $E'$ and $E''$, respectively. Unlike the multi-group cross sections $\sigma_{n,x,k,g}$ considered up to this point, the fission spectrum is a probability distribution and must sum to unity. The outgoing energy is typically much less dependent on the incoming energy for fission than for scattering interactions. As a result, it is common practice to integrate over the incoming neutron energy when computing the multi-group fission spectrum. The fission spectrum may be simplified as $\chi_{n}(\mathbf{r},E)$ with outgoing energy $E$.

Unlike the multi-group cross sections defined up to this point, the multi-group fission spectrum is weighted by the fission production rate rather than the scalar flux. This formulation is intended to preserve the total fission production rate in the multi-group deterministic calculation. In order to mathematically define the multi-group fission spectrum, we denote the microscopic fission cross section as $\sigma_{n,f}(\mathbf{r},E)$ and the average number of neutrons emitted from fission interactions with nuclide $n$ as $\nu_{n}(\mathbf{r},E)$. The multi-group fission spectrum $\chi_{n,k,g}$ is then the probability of fission neutrons emitted into energy group $g$. 

Similar to before, spatial homogenization and energy condensation are used to find the multi-group fission spectrum $\chi_{n,k,g}$ as follows:

$$\chi_{n,k,g'} = \frac{\int_{E_{g'}}^{E_{g'-1}}\mathrm{d}E''\int_{0}^{\infty}\mathrm{d}E'\int_{\mathbf{r} \in V_{k}}\mathrm{d}\mathbf{r}\chi_{n}(\mathbf{r},E'\rightarrow E'')\nu_{n}(\mathbf{r},E')\sigma_{n,f}(\mathbf{r},E')\Phi(\mathbf{r},E')}{\int_{0}^{\infty}\mathrm{d}E'\int_{\mathbf{r} \in V_{k}}\mathrm{d}\mathbf{r}\nu_{n}(\mathbf{r},E')\sigma_{n,f}(\mathbf{r},E')\Phi(\mathbf{r},E')}$$

The fission production-weighted multi-group fission spectrum is computed using OpenMC tallies with both energy in and energy out filters.

This concludes our brief overview on the methodology to compute multi-group cross sections. Let's see it in action now!

## Generate Input Files

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import openmc
import openmc.mgxs as mgxs

import sys
sys.path.append('../..')
from inputs import *

We'll use the simple pin-cell problem for this demonstration.

In [2]:
pincell = PinCellInputSet()

Now we are ready to generate multi-group cross sections! First, let's define an 8-group structure using the built-in `EnergyGroups` class.

In [3]:
# Create 8 energy group structure
groups = mgxs.EnergyGroups([0., 0.058, 0.14, 0.28, 0.625,
                            4.0, 5.53e3, 821.0e3, 20.0e6])

We can now use the `EnergyGroups` object, along with our previously created materials and geometry, to instantiate some `MGXS` objects from the `openmc.mgxs` module. In particular, the following are subclasses of the generic and abstract `MGXS` class:

* `TotalXS`
* `TransportXS`
* `AbsorptionXS`
* `CaptureXS`
* `FissionXS`
* `KappaFissionXS`
* `ScatterXS`
* `ScatterMatrixXS`
* `Chi`
* `ChiPrompt`
* `InverseVelocity`
* `PromptNuFissionXS`

Of course, we are aware that the fission cross section (`FissionXS`) can sometimes be paired with the fission neutron multiplication to become $\nu\sigma_f$. This can be accomodated in to the `FissionXS` class by setting the `nu` parameter to `True` as shown below.

Additionally, scattering reactions (like (n,2n)) can also be defined to take in to account the neutron multiplication to become $\nu\sigma_s$. This can be accomodated in the the transport (`TransportXS`), scattering (`ScatterXS`), and scattering-matrix (`ScatterMatrixXS`) cross sections types by setting the `nu` parameter to `True` as shown below.

These classes provide us with an interface to generate the tally inputs as well as perform post-processing of OpenMC's tally data to compute the respective multi-group cross sections. In this case, let's create the multi-group total, absorption and scattering cross sections with our 2-group structure.

In [15]:
fuel_cell = pincell.geometry.get_cells_by_fill_name('fuel')[0]

In [5]:
# Instantiate a few different sections
total = mgxs.TotalXS(domain=fuel_cell, groups=groups)
absorption = mgxs.AbsorptionXS(domain=fuel_cell, groups=groups)
scattering = mgxs.ScatterXS(domain=fuel_cell, groups=groups)

Each multi-group cross section object stores its tallies in a Python dictionary called `tallies`. We can inspect the tallies in the dictionary for our `Absorption` object as follows. 

In [6]:
absorption.tallies

OrderedDict([('flux', Tally
              	ID             =	10000
              	Name           =	
              	Filters        =	CellFilter, EnergyFilter
              	Nuclides       =	total 
              	Scores         =	['flux']
              	Estimator      =	tracklength), ('absorption', Tally
              	ID             =	10001
              	Name           =	
              	Filters        =	CellFilter, EnergyFilter
              	Nuclides       =	total 
              	Scores         =	['absorption']
              	Estimator      =	tracklength)])

The `Absorption` object includes tracklength tallies for the 'absorption' and 'flux' scores in the 2-group structure in cell 1. Now that each `MGXS` object contains the tallies that it needs, we must add these tallies to a `Tallies` object to generate the "tallies.xml" input file for OpenMC.

In [7]:
# Add total tallies to the tallies file
pincell.tallies.clear()
pincell.tallies += total.tallies.values()

# Add absorption tallies to the tallies file
pincell.tallies += absorption.tallies.values()

# Add scattering tallies to the tallies file
pincell.tallies += scattering.tallies.values()

# Export all input files
pincell.export()

Now we a have a complete set of inputs, so we can go ahead and run our simulation.

In [8]:
# Run OpenMC
openmc.run()


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

0

## Tally Data Processing

Our simulation ran successfully and created statepoint and summary output files. We begin our analysis by instantiating a `StatePoint` object. 

In [9]:
# Load the last statepoint file
sp = openmc.StatePoint('statepoint.50.h5')

In addition to the statepoint file, our simulation also created a summary file which encapsulates information about the materials and geometry. By default, a `Summary` object is automatically linked when a `StatePoint` is loaded. This is necessary for the `openmc.mgxs` module to properly process the tally data.

The statepoint is now ready to be analyzed by our multi-group cross sections. We simply have to load the tallies from the `StatePoint` into each object as follows and our `MGXS` objects will compute the cross sections for us under-the-hood.

In [10]:
# Load the tallies from the statepoint into each MGXS object
total.load_from_statepoint(sp)
absorption.load_from_statepoint(sp)
scattering.load_from_statepoint(sp)

Voila! Our multi-group cross sections are now ready to rock 'n roll!

## Extracting and Storing MGXS Data

Let's first inspect our total cross section by printing it to the screen.

In [11]:
total.print_xs()

Multi-Group XS
	Reaction Type  =	total
	Domain Type    =	cell
	Domain ID      =	10000
	Cross Sections [cm^-1]:
            Group 1 [821000.0   - 20000000.0eV]:	2.74e-01 +/- 8.86e-01%
            Group 2 [5530.0     - 821000.0  eV]:	4.30e-01 +/- 5.72e-01%
            Group 3 [4.0        - 5530.0    eV]:	5.56e-01 +/- 4.73e-01%
            Group 4 [0.625      - 4.0       eV]:	4.22e-01 +/- 1.20e+00%
            Group 5 [0.28       - 0.625     eV]:	4.86e-01 +/- 1.71e+00%
            Group 6 [0.14       - 0.28      eV]:	5.40e-01 +/- 1.78e+00%
            Group 7 [0.058      - 0.14      eV]:	6.08e-01 +/- 1.40e+00%
            Group 8 [0.0        - 0.058     eV]:	8.31e-01 +/- 1.49e+00%





Since the `openmc.mgxs` module uses [tally arithmetic](https://mit-crpg.github.io/openmc/pythonapi/examples/tally-arithmetic.html) under-the-hood, the cross section is stored as a "derived" `Tally` object. This means that it can be queried and manipulated using all of the same methods supported for the `Tally` class in the OpenMC Python API. For example, we can construct a [Pandas](http://pandas.pydata.org/) `DataFrame` of the multi-group cross section data.

In [12]:
df = scattering.get_pandas_dataframe()
df.head(10)

,cell,group in,nuclide,mean,std. dev.
7,10000,1,total,0.263491,0.002329
6,10000,2,total,0.423204,0.002423
5,10000,3,total,0.480060,0.002237
4,10000,4,total,0.384721,0.004603
3,10000,5,total,0.391593,0.006674
2,10000,6,total,0.392966,0.006983
1,10000,7,total,0.394517,0.005523
0,10000,8,total,0.399670,0.005984


Each multi-group cross section object can be easily exported to a variety of file formats, including CSV, Excel, and LaTeX for storage or data processing.

In [13]:
absorption.export_xs_data(filename='absorption-xs', format='excel')

The following code snippet shows how to export all three `MGXS` to the same HDF5 binary data store.

In [14]:
total.build_hdf5_store(filename='mgxs', append=True)
absorption.build_hdf5_store(filename='mgxs', append=True)
scattering.build_hdf5_store(filename='mgxs', append=True)

## Comparing MGXS with Tally Arithmetic

Finally, we illustrate how one can leverage OpenMC's [tally arithmetic](https://mit-crpg.github.io/openmc/pythonapi/examples/tally-arithmetic.html) data processing feature with `MGXS` objects. The `openmc.mgxs` module uses tally arithmetic to compute multi-group cross sections with automated uncertainty propagation. Each `MGXS` object includes an `xs_tally` attribute which is a "derived" `Tally` based on the tallies needed to compute the cross section type of interest. These derived tallies can be used in subsequent tally arithmetic operations. For example, we can use tally artithmetic to confirm that the `TotalXS` is equal to the sum of the `AbsorptionXS` and `ScatterXS` objects.

In [15]:
# Use tally arithmetic to compute the difference between the total, absorption and scattering
difference = total.xs_tally - absorption.xs_tally - scattering.xs_tally

# The difference is a derived tally which can generate Pandas DataFrames for inspection
difference.get_pandas_dataframe()

,cell,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,10000,0.000,5.800000e-02,total,(((total / flux) - (absorption / flux)) - (sca...,0.000000e+00,0.015175
1,10000,0.058,1.400000e-01,total,(((total / flux) - (absorption / flux)) - (sca...,2.220446e-16,0.010585
2,10000,0.140,2.800000e-01,total,(((total / flux) - (absorption / flux)) - (sca...,2.220446e-16,0.012141
3,10000,0.280,6.250000e-01,total,(((total / flux) - (absorption / flux)) - (sca...,1.665335e-16,0.010771
4,10000,0.625,4.000000e+00,total,(((total / flux) - (absorption / flux)) - (sca...,1.665335e-16,0.006857
5,10000,4.000,5.530000e+03,total,(((total / flux) - (absorption / flux)) - (sca...,3.330669e-16,0.003511
6,10000,5530.000,8.210000e+05,total,(((total / flux) - (absorption / flux)) - (sca...,3.330669e-16,0.003452
7,10000,821000.000,2.000000e+07,total,(((total / flux) - (absorption / flux)) - (sca...,2.220446e-16,0.003369


Similarly, we can use tally arithmetic to compute the ratio of `AbsorptionXS` and `ScatterXS` to the `TotalXS`.

In [16]:
# Use tally arithmetic to compute the absorption-to-total MGXS ratio
absorption_to_total = absorption.xs_tally / total.xs_tally

# The absorption-to-total ratio is a derived tally which can generate Pandas DataFrames for inspection
absorption_to_total.get_pandas_dataframe()

,cell,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,10000,0.000,5.800000e-02,total,((absorption / flux) / (total / flux)),0.519239,0.010935
1,10000,0.058,1.400000e-01,total,((absorption / flux) / (total / flux)),0.351600,0.006968
2,10000,0.140,2.800000e-01,total,((absorption / flux) / (total / flux)),0.272140,0.006825
3,10000,0.280,6.250000e-01,total,((absorption / flux) / (total / flux)),0.193745,0.004753
4,10000,0.625,4.000000e+00,total,((absorption / flux) / (total / flux)),0.088373,0.001554
5,10000,4.000,5.530000e+03,total,((absorption / flux) / (total / flux)),0.137221,0.001318
6,10000,5530.000,8.210000e+05,total,((absorption / flux) / (total / flux)),0.015156,0.000129
7,10000,821000.000,2.000000e+07,total,((absorption / flux) / (total / flux)),0.039825,0.000544


In [17]:
# Use tally arithmetic to compute the scattering-to-total MGXS ratio
scattering_to_total = scattering.xs_tally / total.xs_tally

# The scattering-to-total ratio is a derived tally which can generate Pandas DataFrames for inspection
scattering_to_total.get_pandas_dataframe()

,cell,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,10000,0.000,5.800000e-02,total,((scatter / flux) / (total / flux)),0.480761,0.010150
1,10000,0.058,1.400000e-01,total,((scatter / flux) / (total / flux)),0.648400,0.012835
2,10000,0.140,2.800000e-01,total,((scatter / flux) / (total / flux)),0.727860,0.018282
3,10000,0.280,6.250000e-01,total,((scatter / flux) / (total / flux)),0.806255,0.019447
4,10000,0.625,4.000000e+00,total,((scatter / flux) / (total / flux)),0.911627,0.015441
5,10000,4.000,5.530000e+03,total,((scatter / flux) / (total / flux)),0.862779,0.005726
6,10000,5530.000,8.210000e+05,total,((scatter / flux) / (total / flux)),0.984844,0.007972
7,10000,821000.000,2.000000e+07,total,((scatter / flux) / (total / flux)),0.960175,0.012017


Lastly, we sum the derived scatter-to-total and absorption-to-total ratios to confirm that they sum to unity.

In [18]:
# Use tally arithmetic to ensure that the absorption- and scattering-to-total MGXS ratios sum to unity
sum_ratio = absorption_to_total + scattering_to_total

# The scattering-to-total ratio is a derived tally which can generate Pandas DataFrames for inspection
sum_ratio.get_pandas_dataframe()

,cell,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,10000,0.000,5.800000e-02,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.014920
1,10000,0.058,1.400000e-01,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.014605
2,10000,0.140,2.800000e-01,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.019514
3,10000,0.280,6.250000e-01,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.020019
4,10000,0.625,4.000000e+00,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.015519
5,10000,4.000,5.530000e+03,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.005876
6,10000,5530.000,8.210000e+05,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.007973
7,10000,821000.000,2.000000e+07,total,(((absorption / flux) / (total / flux)) + ((sc...,1.0,0.012029


## Energy Group Collapse


Next, we illustate how one can easily take multi-group cross sections and condense them down to a coarser energy group structure. The `MGXS` class includes a `get_condensed_xs(...)` method which takes an `EnergyGroups` parameter with a coarse(r) group structure and returns a new `MGXS` condensed to the coarse groups. We illustrate this process below using the 2-group structure created earlier.

In [19]:
coarse_groups = mgxs.EnergyGroups([0.0, 0.625, 20.0e6])

coarse_scattering = scattering.get_condensed_xs(coarse_groups)

coarse_scattering.print_xs()

Multi-Group XS
	Reaction Type  =	scatter
	Domain Type    =	cell
	Domain ID      =	10000
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 20000000.0eV]:	3.88e-01 +/- 3.59e-01%
            Group 2 [0.0        - 0.625     eV]:	3.96e-01 +/- 8.13e-01%





## Cross Section Libraries

Often we want to get multiple multi-group cross sections for multiple domains. In this case, it is easier to use the `mgxs.Library` class to help us generate all the tallies necessary. To create a `Library` we have to pass it a copy of our geometry.

In [4]:
mgxs_lib = mgxs.Library(pincell.geometry)

Now we need to assign energy groups that will be used for our library.

In [5]:
mgxs_lib.energy_groups = groups

Then we tell it what cross sections we want:

In [6]:
mgxs_lib.mgxs_types = ['total', 'fission', 'scatter matrix', 'chi']

And which domains to compute cross sections for:

In [7]:
mgxs_lib.domain_type = 'cell'
mgxs_lib.domains = pincell.geometry.get_all_cells().values()

If we want cross sections for each nuclide, we can also specify that on a library-basis.

In [8]:
mgxs_lib.by_nuclide = True

When we done, we can create all the tallies necessary by calling `build_library()`.

In [9]:
mgxs_lib.build_library()

Now we need to add all the tallies to our `Tallies` object. One problem is that if we simply add all the tallies necessary for many domains and cross section types, we could end up with > 100 distinct `Tally` objects. The overhead to tally in OpenMC scales as $O(N)$ for $N$ tallies, which can become a bottleneck for large tally datasets. To compensate for this, the Python API's `Tally`, `Filter` and `Tallies` classes allow for the smart *merging* of tallies when possible. The `Library` class supports this runtime optimization with the use of the optional `merge` paramter (`False` by default) for the `Library.add_to_tallies_file(...)` method, as shown below.

In [10]:
pincell.tallies.clear()
mgxs_lib.add_to_tallies_file(pincell.tallies, merge=True)
pincell.export()

At this point, we can run OpenMC and pull in the tally results using `Library.load_from_statepoint` which works exactly like `MGXS.load_from_statepoint`.

In [11]:
openmc.run(output=False)

0

In [13]:
sp = openmc.StatePoint('statepoint.50.h5')
mgxs_lib.load_from_statepoint(sp)

Our results have been loaded in now. To get a specific MGXS, we can use the `Library.get_mgxs()` method which takes the domain and MGXS type as arguments. Let's say we want to look at the fission cross section in fuel.

In [16]:
fuel_fission = mgxs_lib.get_mgxs(fuel_cell, 'fission')
fuel_fission

If we wanted to, say, look at the U238 cross section in particular. We can use `print_xs` or `get_pandas_dataframe` to get that one in particle.

In [17]:
fuel_fission.print_xs(nuclides=['U238'], xs_type='micro')

Multi-Group XS
	Reaction Type  =	fission
	Domain Type    =	cell
	Domain ID      =	10000
	Nuclide        =	U238
	Cross Sections [barns]:
            Group 1 [821000.0   - 20000000.0eV]:	3.80e-01 +/- 1.09e+00%
            Group 2 [5530.0     - 821000.0  eV]:	4.78e-04 +/- 1.40e+00%
            Group 3 [4.0        - 5530.0    eV]:	1.98e-04 +/- 1.52e+01%
            Group 4 [0.625      - 4.0       eV]:	2.63e-06 +/- 1.21e+00%
            Group 5 [0.28       - 0.625     eV]:	4.31e-06 +/- 1.71e+00%
            Group 6 [0.14       - 0.28      eV]:	6.20e-06 +/- 1.78e+00%
            Group 7 [0.058      - 0.14      eV]:	9.14e-06 +/- 1.40e+00%
            Group 8 [0.0        - 0.058     eV]:	1.65e-05 +/- 1.49e+00%





/home/romano/openmc/openmc/tallies.py:1875: RuntimeWarning: invalid value encountered in true_divide
  self_rel_err = data['self']['std. dev.'] / data['self']['mean']
